In [2]:
import pandas as pd

In [18]:
# Load data from CSV file
df = pd.read_csv('Occupations.csv', delimiter=';')

In [19]:
df.shape

(1016, 2)

In [20]:
df.head()

,O*NET-SOC Code,Occupation
0,11-1011.00,Chief Executives
1,11-1011.03,Chief Sustainability Officers
2,11-1021.00,General and Operations Managers
3,11-1031.00,Legislators
4,11-2011.00,Advertising and Promotions Managers


In [21]:
# Checking for missing values
df.isnull().sum()   

O*NET-SOC Code    0
Occupation        0
dtype: int64

## Crosswalks

### O*NET --> SOC2019 --> SOC2010 --> ISCO 2008

In [23]:
import pandas as pd

# -------------------------
# 1. Load the three files
# -------------------------
occ = pd.read_csv("Occupations.csv", sep=";")
cw_10_19 = pd.read_csv("2010_to_2019_Crosswalk.csv", sep=";")
cw_isco_soc = pd.read_csv("ISCO_SOC_Crosswalk.csv", sep=";")

# -------------------------
# 2. Basic cleaning (strip spaces)
# -------------------------
for df in (occ, cw_10_19, cw_isco_soc):
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].str.strip()

# Quick check of the key columns (not required, but useful to print once)
print("Occupations codes example:", occ["O*NET-SOC Code"].head().tolist())
print("2019 codes in 2010-2019 crosswalk:", cw_10_19["soc2019_code"].head().tolist())
print("2010 codes in ISCO crosswalk:", cw_isco_soc["soc2010_code"].head().tolist())

# ---------------------------------------------------
# 3. STEP 1 – Map SOC 2019 (O*NET) → SOC 2010
#    Occupations.O*NET-SOC Code  ==  cw_10_19.soc2019_code
# ---------------------------------------------------
occ_merged = occ.merge(
    cw_10_19[["soc2019_code", "soc2010_code"]],
    left_on="O*NET-SOC Code",
    right_on="soc2019_code",
    how="left"
)

print("Share of rows without SOC 2010 mapping:",
      occ_merged["soc2010_code"].isna().mean())

# ---------------------------------------------------
# 4. Create a 6-digit SOC2010 code (no decimal)
#    to match the ISCO crosswalk (which uses 11-1011)
# ---------------------------------------------------
occ_merged["soc2010_6"] = occ_merged["soc2010_code"].astype(str).str.split(".").str[0]

# Make sure SOC2010 in ISCO crosswalk is string and stripped
cw_isco_soc["soc2010_code"] = cw_isco_soc["soc2010_code"].astype(str).str.strip()

# ---------------------------------------------------
# 5. STEP 2 – Map SOC 2010 → ISCO 2008
#    occ_merged.soc2010_6  ==  cw_isco_soc.soc2010_code
# ---------------------------------------------------
final = occ_merged.merge(
    cw_isco_soc[["soc2010_code", "isco08_code", "isco08_title"]],
    left_on="soc2010_6",
    right_on="soc2010_code",
    how="left",
    suffixes=("", "_isco")
)

print("Share of rows without ISCO mapping:",
      final["isco08_code"].isna().mean())

# ---------------------------------------------------
# 6. Optional: tidy up columns and save
# ---------------------------------------------------
# Drop some helper columns if you don’t need them
final = final.drop(columns=["soc2010_code_isco"])  # this is the one from ISCO file

# Save result
final.to_csv("Occupations_with_SOC2010_ISCO.csv", index=False)

print("Done! Shape of final file:", final.shape)
print(final.head())


Occupations codes example: ['11-1011.00', '11-1011.03', '11-1021.00', '11-1031.00', '11-2011.00']
2019 codes in 2010-2019 crosswalk: ['11-1011.00', '11-1011.03', '11-1021.00', '11-1031.00', '11-2011.00']
2010 codes in ISCO crosswalk: ['11-1011', '11-1011', '11-1011', '11-1021', '11-1021']
Share of rows without SOC 2010 mapping: 0.003424657534246575
Share of rows without ISCO mapping: 0.002727768685215494
Done! Shape of final file: (1833, 7)
  O*NET-SOC Code                     Occupation soc2019_code soc2010_code  \
0     11-1011.00               Chief Executives   11-1011.00   11-1011.00   
1     11-1011.00               Chief Executives   11-1011.00   11-1011.00   
2     11-1011.00               Chief Executives   11-1011.00   11-1011.00   
3     11-1011.03  Chief Sustainability Officers   11-1011.03   11-1011.03   
4     11-1011.03  Chief Sustainability Officers   11-1011.03   11-1011.03   

  soc2010_6  isco08_code                              isco08_title  
0   11-1011       1112.

### ILO 4 digit codes --> AI-exposure codes

In [25]:
import pandas as pd

# ----------------------------------------------------
# 1. Load both files
# ----------------------------------------------------
occ1 = pd.read_csv("Occupations_with_SOC2010_ISCO.csv")
isco = pd.read_csv("ISCO 4digit CODE with AI-exposure score.csv")

# ----------------------------------------------------
# 2. Clean ISCO codes properly
# ----------------------------------------------------

# Fix occupations ISCO codes: remove ".0" and convert to 4-digit strings
occ1["isco08_code"] = (
    occ1["isco08_code"]
    .astype(str)
    .str.replace(".0", "", regex=False)
    .str.strip()
)

# Exposure file code column
isco["4-digit code"] = (
    isco["4-digit code"]
    .astype(str)
    .str.replace(".0", "", regex=False)
    .str.strip()
)

# ----------------------------------------------------
# 3. MERGE Exposure + Mean
# ----------------------------------------------------
merged = occ1.merge(
    isco[["4-digit code", "Exposure", "Mean"]],
    left_on="isco08_code",
    right_on="4-digit code",
    how="left"
)

# ----------------------------------------------------
# 4. Save final dataset
# ----------------------------------------------------
merged.to_csv("ILO_Occupations_with_exposure_score.csv", index=False)

print("Done! Created: ILO_Occupations_with_exposure_score.csv")
print(merged[["isco08_code", "Exposure", "Mean"]].head())


Done! Created: ILO_Occupations_with_exposure_score.csv
  isco08_code          Exposure  Mean
0        1112  Minimal Exposure  0.38
1        1113       Not Exposed  0.23
2        1120  Minimal Exposure  0.38
3        1112  Minimal Exposure  0.38
4        1113       Not Exposed  0.23


In [26]:
print(merged.head())

  O*NET-SOC Code                     Occupation soc2019_code soc2010_code  \
0     11-1011.00               Chief Executives   11-1011.00   11-1011.00   
1     11-1011.00               Chief Executives   11-1011.00   11-1011.00   
2     11-1011.00               Chief Executives   11-1011.00   11-1011.00   
3     11-1011.03  Chief Sustainability Officers   11-1011.03   11-1011.03   
4     11-1011.03  Chief Sustainability Officers   11-1011.03   11-1011.03   

  soc2010_6 isco08_code                              isco08_title  \
0   11-1011        1112               Senior government officials   
1   11-1011        1113  Traditional chiefs and heads of villages   
2   11-1011        1120   Managing directors and chief executives   
3   11-1011        1112               Senior government officials   
4   11-1011        1113  Traditional chiefs and heads of villages   

  4-digit code          Exposure  Mean  
0         1112  Minimal Exposure  0.38  
1         1113       Not Exposed  0.23  